# Projeto de sistema de transmissão para eixos deslocados em 90°

Disciplina: TMEC038 Elementos de Máquinas II

Alunos:
- Marcos Augusto Belizario
- Murasson Obil

## 1. Importando pacotes necessários

In [36]:
import numpy as np

from abc import ABC, abstractmethod

## 2. Definindo as caracteristicas do sistema de transmissão

In [37]:
# Caracteristicas da transmissão
rpm_out = 900.0 / 4.0
power_out = 8.0 * 735.5
ratio = 4.0
life_cycle = 1.0e7
reliability = 0.92

<img src="./tab-13-2.png" alt="Tabela 13.2" style="height: 400px;"/>
<img src="./fig-13-7.png" alt="Figura 13.7" style="height: 400px;"/>

In [38]:
# Caracteristicas das engrenagens
alpha = 20.0
qv = 9
qm = 2
kmb = 1.1
k0 = 2.0
kx = 1.0

sigma_esc = 125.4e6
cs = 1.0

## 3. Cálculo dos parâmetros geométricos e de resistência dos materiais

In [39]:
class Gear(ABC):
    def __init__(self, z, m, b, alpha, qv):
        self.z = z
        self.m = m
        self.b = b
        self.alpha = alpha
        self.qv = qv
        
    @property
    @abstractmethod
    def d(self):
        raise NotImplementedError()
    
    @abstractmethod
    def ks(self):
        raise NotImplementedError()

    @abstractmethod
    def km(self):
        raise NotImplementedError()

    @abstractmethod
    def sigma_agma(self, k0, j, torque, rpm):
        raise NotImplementedError()
    
    @staticmethod
    @abstractmethod
    def yn(n):
        raise NotImplementedError()
    
    @abstractmethod
    def csf(self, st, n, temp, r, k0, j, torque, rpm):
        raise NotImplementedError()
    
    @property
    def dp(self):
        return self.z * self.m

    def tangencial_force(self, torque):
        return 2.0 * torque / self.d

    def tangencial_velocity(self, rpm):
        return self.d * rpm * np.pi / 60.0
    
    def kv(self, rpm):
        aux_b = 0.25 * (12.0 - qv) ** (2.0 / 3.0)
        aux_a = 50.0 + 56.0 * (1.0 - aux_b)
        return (1.0 + np.sqrt(200.0 * self.tangencial_velocity(rpm)) / aux_a) ** aux_b
    
    @staticmethod
    def yt(temp):
        if temp < 120.0:
            return 1.0
        else:
            return (273.0 + temp) / 393.0
        
    @staticmethod
    def yz(r):
        if r < 0.99:
            return 0.658 - 0.0759 * np.log(1.0 - r)
        else:
            return 0.5 - 0.109 * np.log(1.0 - r)


class ConicalGear(Gear):
    def __init__(self, z, m, b, alpha, qv, gamma, kx):
        super().__init__(z, m, b, alpha, qv)
        self.gamma = gamma
        self.kx = kx
        
    @property
    def dm(self):
        return self.dp - self.b * np.sin(self.gamma)
    
    @property
    def d(self):
        return self.dm
    
    def ks(self):
        if self.m < 1.6e-3:
            return 0.5
        elif self.m < 50.0e-3:
            return 0.4867 + 8.339 * self.m
        
    def km(self):
        return kmb + 5.6 * self.b ** 2.0

    def sigma_agma(self, k0, j, torque, rpm):
        force_factors = k0 * self.kv(rpm) * self.ks() * self.km()
        geometry_factors = self.kx * j
        return force_factors * self.tangencial_force(torque) / (geometry_factors * self.b * self.m)
    
    @staticmethod
    def yn(n):
        if n < 3.0e6:
            return 6.1514 * n ** -0.1192
        else:
            return 1.683 * n ** -0.0323

    def csf(self, st, n, temp, r, k0, j, torque, rpm):
        # return st * self.yn(n) / (Gear.yt(temp) * Gear.yz(r) * self.sigma_agma(k0, j, torque, rpm))
        return st * 0.896 / (Gear.yt(temp) * Gear.yz(r) * self.sigma_agma(k0, j, torque, rpm))

In [40]:
def calc_states(power=None, torque=None, rpm=None):
    if power is None:
        power = torque * (rpm * np.pi / 30.0)
    elif torque is None:
        torque = power / (rpm * np.pi / 30)
    elif rpm is None:
        rpm = (30.0 / np.pi) * power / torque

    return power, torque, rpm


def form_factor(z, alpha=20.0):
    z_data = np.array([10, 11, 12, 13, 14, 
                  15, 16, 17, 18, 19, 
                  20, 22, 24, 26, 28, 
                  30, 32, 34, 36, 38, 
                  40, 45, 50, 55, 60, 
                  65, 70, 75, 80, 90, 
                  100, 150, 200, 300], dtype=np.int32)
    
    y_145 = np.array([0.176, 0.192, 0.210, 0.223, 0.236, 
                      0.245, 0.255, 0.264, 0.270, 0.277,
                      0.283, 0.292, 0.302, 0.308, 0.314, 
                      0.318, 0.322, 0.325, 0.329, 0.332,
                      0.336, 0.340, 0.346, 0.352, 0.355, 
                      0.358, 0.360, 0.361, 0.363, 0.366, 
                      0.368, 0.375, 0.378, 0.382], dtype=np.float64)
    
    y_200 = np.array([0.201, 0.226, 0.245, 0.264, 0.276, 
                      0.289, 0.295, 0.302, 0.308, 0.314, 
                      0.320, 0.330, 0.337, 0.344, 0.352, 
                      0.358, 0.364, 0.370, 0.377, 0.383, 
                      0.389, 0.399, 0.408, 0.415, 0.421, 
                      0.425, 0.429, 0.433, 0.436, 0.442, 
                      0.446, 0.458, 0.463, 0.471], dtype=np.float64)
    
    y_145_pred = np.interp(z, z_data, y_145)
    y_200_pred = np.interp(z, z_data, y_200)
    y_pred = np.interp(alpha, [14.5, 20.0], [y_145_pred, y_200_pred])
    
    return y_pred
    
def lewis_equation(m, z, alpha, k0, ft, sigma_esc, cs=1.0):
    y = form_factor(z, alpha)
    return cs * (k0 * ft) / (m * y * sigma_esc)  # return b


def generate_gears(m0, z10, alpha, qv, k0, kx, sigma_esc, cs, ratio, torque, rtol=1.0e-6, niter=100):
    b0 = 1.0e-3
    
    pinion = ConicalGear(z10, m0, b0, alpha, qv, np.arctan(1.0 / ratio), kx)
    crown = ConicalGear(round(ratio * pinion.z), m0, b0, alpha, qv, np.arctan(ratio), kx)

    for _ in range(niter):
        b0 = lewis_equation(m0, z10, alpha, k0, pinion.tangencial_force(torque), sigma_esc, cs)
        pinion.b = b0
        crown.b = b0

    return pinion, crown

## 4. Aplicando o equacionamento ao estudo de caso

### 4.1. Obtendo os parâmetros iniciais a partir da Equação de Lewis

Para evitar interferência, temos que z1 >= 15, e para obter a quantidade de dentes na coroa seja inteira com uma relação de transmissão de 1.5, temos que z1 deve ser divísivel por 2.
Portanto, o menor número de dentes para o pinhão é z1 = 16.

In [41]:
m0 = 2.5e-3
z10 = 15

power1, torque1, rpm1 = calc_states(power_out, None, rpm_out * ratio)
power2, torque2, rpm2 = calc_states(power_out, None, rpm_out)

pinion, crown = generate_gears(m0, z10, alpha, qv, k0, kx, sigma_esc, cs, ratio, torque1)

In [42]:
pinion.b = 25.0e-3
crown.b = 25.0e-3

print(f"Pinhao: z = {pinion.z} | m = {pinion.m * 1.0e3:.2f} mm | b = {pinion.b * 1.0e3:.2f} mm | dp = {pinion.dp * 1.0e3:.2f} mm | dm = {pinion.dm * 1.0e3:.2f} mm")
print(f" Coroa: z = {crown.z} | m = {crown.m * 1.0e3:.2f} mm | b = {crown.b * 1.0e3:.2f} mm | dp = {crown.dp * 1.0e3:.2f} mm | dm = {crown.dm * 1.0e3:.2f} mm")

Pinhao: z = 15 | m = 2.50 mm | b = 25.00 mm | dp = 37.50 mm | dm = 31.44 mm
 Coroa: z = 60 | m = 2.50 mm | b = 25.00 mm | dp = 150.00 mm | dm = 125.75 mm


<img src="./fig-13-6.png" alt="Figura 13.6" style="height: 400px;"/>

In [43]:
j1 = 0.24
pinion_sigma_agma = pinion.sigma_agma(k0, j1, torque1, rpm1)
pinion_csf = pinion.csf(sigma_esc, life_cycle, 70.0, reliability, k0, j1, torque1, rpm1)
print(f"Pinhao: sigma_agma = {pinion_sigma_agma * 1.0e-6:.3f} MPa | CSf = {pinion_csf:.3f}")

j2 = 0.179
crown_sigma_agma = crown.sigma_agma(k0, j2, torque2, rpm2)
crown_csf = crown.csf(sigma_esc, life_cycle / ratio, 70.0, reliability, k0, j2, torque2, rpm2)
print(f"Coroa: sigma_agma = {crown_sigma_agma * 1.0e-6:.3f} MPa | CSf = {crown_csf:.3f}")

Pinhao: sigma_agma = 329.468 MPa | CSf = 0.401
Coroa: sigma_agma = 441.744 MPa | CSf = 0.299
